In [1]:
source('cascade_helper.R')


In [2]:
p_thr <- list(1e-7, 1e-7, 5e-8)
names(p_thr) <- c('protein-truncating', 'protein-altering', 'non-coding')
p_thr_df <- as.data.frame(p_thr) %>% gather(Csq, p_thr) %>% mutate(Csq = str_replace_all(Csq, '\\.', '-'))


In [3]:
outd <- file.path(getwd(), 'out')


In [4]:
annot.tbl <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz'
annot.arr <- read_annot_arr(annot.tbl)


In [5]:
annot.arr %>% count(Csq)


Csq,n
<chr>,<int>
non-coding,669033
protein-altering,87407
protein-truncating,27816


In [6]:
traits <- fread('cascade.input.files.tsv')


In [7]:
dim(traits)


[1] 35  3

In [8]:
traits %>% head(3)

phe,array_meta,imp_meta
<chr>,<chr>,<chr>
Alanine_aminotransferase,/oak/stanford/groups/mrivas/projects/biomarkers_rivas/meta_flipfixed/METAANALYSIS_Alanine_aminotransferase_1.tbl.gz,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Alanine_aminotransferase.sumstats.tsv.gz
Albumin,/oak/stanford/groups/mrivas/projects/biomarkers_rivas/meta_flipfixed/METAANALYSIS_Albumin_1.tbl.gz,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Albumin.sumstats.tsv.gz
Alkaline_phosphatase,/oak/stanford/groups/mrivas/projects/biomarkers_rivas/meta_flipfixed/METAANALYSIS_Alkaline_phosphatase_1.tbl.gz,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Alkaline_phosphatase.sumstats.tsv.gz


In [9]:
######################
read_sumstats <- function(file, pval_thr){
    df <- fread(
        cmd=paste0(
            'zcat ', file, ' | sed -e "s/^#//g"')
    ) %>%
    rename('P' = 'P-value') %>%
    mutate(P = as.numeric(P)) %>%
    filter(P <= pval_thr)    
}

In [10]:
df <- read_sumstats('/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Urea.sumstats.tsv.gz', 5e-8)


In [13]:
df %>% head()

MarkerName,REF,ALT,Effect,StdErr,P,Direction,HetISq,HetChiSq,HetDf,HetPVal,MAF,MACH_R2
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
3:136255483_C_T,C,T,-0.0293,0.0028,8.538e-26,---+,61.5,7.791,3,0.05053,0.2379690,0.996862
2:148562909_C_A,C,A,0.0148,0.0026,8.376e-09,++++,0.0,0.333,3,0.95370,0.3093460,0.997099
9:15603066_A_G,A,G,-0.0144,0.0024,1.478e-09,----,0.0,1.421,3,0.70070,0.4523890,0.996399
5:34449786_G_T,G,T,0.0165,0.0025,6.337e-11,++++,0.0,1.093,3,0.77880,0.3342910,0.999301
5:176807191_C_T,C,T,0.0248,0.0041,1.551e-09,+++-,21.1,3.800,3,0.28390,0.0970857,0.945972
3:135804550_T_A,T,A,0.0159,0.0028,1.893e-08,++++,0.0,0.211,3,0.97580,0.2346450,0.990066


In [ ]:
read_imp_sumstats <- function(file, pval_thr){
    df <- fread(
        cmd=paste0(
            'zcat ', file, ' | sed -e "s/^#//g"')
    ) %>%
    rename('P' = 'P-value') %>%
    mutate(P = as.numeric(P)) %>%
    filter(P <= pval_thr)    
}


In [7]:
array_anno_df <- traits %>% 
read_array_sumstats_all(p_thr_df %>% select(p_thr) %>% max()) %>%
annotate_array_df(annot.arr, p_thr_df)


In [8]:
array_anno_df %>%
dim()

[1] 83228    24

In [9]:
array_anno_df %>%
fwrite(file.path(outd, 'cascade.array.hits.tsv'), sep='\t')


In [16]:
array_anno_df %>% 
filter(Csq != 'non-coding') %>%
select(Csq, ld_indep, is_rare, is_outside_of_MHC, ID) %>%
unique() %>%
count(is_outside_of_MHC, Csq, ld_indep, is_rare)

is_outside_of_MHC,Csq,ld_indep,is_rare,n
<lgl>,<chr>,<lgl>,<lgl>,<int>
FALSE,protein-altering,FALSE,FALSE,294
FALSE,protein-altering,FALSE,TRUE,4
FALSE,protein-altering,TRUE,FALSE,51
FALSE,protein-altering,TRUE,TRUE,9
FALSE,protein-truncating,FALSE,FALSE,15
FALSE,protein-truncating,FALSE,TRUE,1
FALSE,protein-truncating,TRUE,TRUE,1
TRUE,protein-altering,FALSE,FALSE,2148
TRUE,protein-altering,FALSE,TRUE,34
